# Importación de Librerías

In [1]:
import pandas as pd
import numpy as np
import re
import sklearn
# import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

#import plotly.offline as py
#import plotly.graph_objs as go
#import plotly.tools as tls

import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier,\
                             GradientBoostingClassifier, ExtraTreesClassifier)

from sklearn.svm import SVC
from sklearn.cross_validation import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Set de Entrenamiento y Test Final

In [2]:
train = pd.read_csv('modelo.csv')
test_final = pd.read_csv('test_final.csv')

In [3]:
si = train['se_postulo'] == 1
no = train['se_postulo'] == 0

train_si = train[si].sample(200000)
train_no = train[no].sample(200000)

train_final = pd.merge(train_si, train_no, how='outer')

x_train = np.array(train_final[['sexo', 'orden_estudio', 'tipo_de_trabajo', 'nivel_laboral']])
y_train = np.array(train_final['se_postulo'])

x_test = np.array(test_final[['sexo', 'orden_estudio', 'tipo_de_trabajo', 'nivel_laboral']])

### Análisis de la Correlación de Pearson entre los distintos features

In [15]:
# Heatmap de la Correlación de Pearson
# colormap = plt.cm.RdBu
# plt.figure(figsize=(10,10))
# plt.title('Features - Correlación de Pearson', y=1.05, size=15)
# sns.heatmap(train.astype(float).corr(),linewidths=0.1,vmax=1.0, 
#            square=True, cmap=colormap, linecolor='white', annot=True)


# Definición de algunas variables globales

In [4]:
id_aviso_postulante = test_final['id']

ntrain = x_train.shape[0] # Cantidad de registros totales del set de entrenamiento

ntest = x_test.shape[0] # Cantidad de registros totales del set de pruebas

SEED = 0 # Seed

NFOLDS = 5 # Cantidad de bloques a particionar para KFold

kfold = KFold(ntrain, n_folds=NFOLDS, random_state=SEED) # KFold

# Creación de la clase SklearnPadre
Lo que realizamos a continuación es la creación de una clase que posee los métodos comunes de los clasificadores de Sklearn que utilizaremos en el presente trabajo. De esta forma, ahorramos líneas de código y evitamos redundancia a la hora de utilizar los métodos.

El inicializador recibe un clasificador de Sklearn, un seed y los parámetros necesarios del clasificador en cuestión.

In [5]:
class SklearnPadre(object):
    def __init__(self, clasif, seed=0, parametros=None):
        parametros['random_state'] = seed
        self.clasif = clasif(**parametros)

    def train(self, x_train, y_train):
        self.clasif.fit(x_train, y_train)

    def predict(self, x):
        return self.clasif.predict(x)
    
    def fit(self, x, y):
        return self.clasif.fit(x,y)
    
    def feature_importances(self, x, y):
        print(self.clasif.fit(x,y).feature_importances_)

# Predicciones OOF (Out Of Fold)

EXPLICAR UN POCO PARA QUÉ SIRVE ESTA FUNCIÓN

In [6]:
def get_oof(clasif, x_train, y_train, x_test):
    oof_train = np.zeros(ntrain)
    oof_test = np.zeros(ntest)
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kfold):
        x_train_i = x_train[train_index]
        y_train_i = y_train[train_index]
        x_test_i = x_train[test_index]

        clasif.train(x_train_i, y_train_i)

        oof_train[test_index] = clasif.predict(x_test_i)
        oof_test_skf[i, :] = clasif.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1) 

# Modelos de Primer Nivel

A continuación llevaremos a cabo nuestro primer nivel de clasificación mediante el uso de algunos clasificadores de la librería Sklearn:

- Random Forest
- Extra Trees
- Adaptive Boosting
- Gradient Boosting
- SVM

### Definición de los hiper-parámetros necesarios para cada algoritmo

In [7]:
'''RANDOM FOREST'''
# n_estimators      --> cantidad de árboles en el random forest
# max_features      --> cant_maxima de features a considerar antes de realizar un split
# max_depth         --> profundidad máxima del árbol
# min_samples_split --> minimo de muestras necesario para realizar un split
# min_samples_leaf  --> minimo de muestras necesario para que se trate de un nodo hoja
# oof_score         --> True si se utilizan las muestras OOB para estimar la precisión
# n_jobs            --> -1 = numero de cores (para correr en paralelo fit y predict)
# warm_start        --> True si se utiliza la solución previa del fit y añade más estimadores al ensamble
rf_params = {'n_estimators': 100,
             'max_features': 'sqrt',
             'max_depth': 5,
             'min_samples_split': 2,
             'min_samples_leaf': 2,
             'bootstrap': True,
             'oob_score': True,
             'n_jobs': -1,
             'warm_start': True
             }


'''EXTRA TREES'''
et_params = {'n_estimators': 100,
             'max_features': 'sqrt',
             'max_depth': 5,
             'min_samples_split': 2,
             'min_samples_leaf': 2,
             'bootstrap': True,
             'oob_score': True,
             'n_jobs': -1,
             'warm_start': True
             }


'''ADAPTIVE BOOSTING'''
# learning_rate --> en cuánto se reduce la tasa de contribución de cada clasificador
ab_params = {'n_estimators': 100,
             'learning_rate': 0.75
             }


'''GRADIENT BOOSTING'''
gb_params = {'learning_rate': 0.1,
             'n_estimators': 100,
             'max_depth': 4,
             'min_samples_split': 2,
             'min_samples_leaf': 2,
             'max_features': 'sqrt',
             'warm_start': True 
             }


'''SVM'''
# C      --> costo por clasificar mal un punto
# kernel --> kernel a utilizar en el algoritmo
# gamma  --> parámetro gamma para el kernel RBF
svm_params = {'C': 0.5,
              'kernel': 'rbf',
              'gamma': 'auto'
              }

### Creación de los clasificadores según los hiper-parámetros definidos

In [8]:
random_forest = SklearnPadre(clasif=RandomForestClassifier, seed=SEED, parametros=rf_params)

extra_trees = SklearnPadre(clasif=ExtraTreesClassifier, seed=SEED, parametros=et_params)

ada_boost = SklearnPadre(clasif=AdaBoostClassifier, seed=SEED, parametros=ab_params)

grad_boost = SklearnPadre(clasif=GradientBoostingClassifier, seed=SEED, parametros=gb_params)

svm = SklearnPadre(clasif=SVC, seed=SEED, parametros=svm_params)

### Output de los modelos de primer nivel

In [9]:
# Predicciones del set de entrenamiento y de pruebas con OOF.
# Estos resultados base serán utilizados como nuevos features en el modelo de segundo nivel.
rf_oof_train, rf_oof_test = get_oof(random_forest,x_train, y_train, x_test)

et_oof_train, et_oof_test = get_oof(extra_trees, x_train, y_train, x_test)

ab_oof_train, ab_oof_test = get_oof(ada_boost, x_train, y_train, x_test) 

gb_oof_train, gb_oof_test = get_oof(grad_boost,x_train, y_train, x_test)

svm_oof_train, svm_oof_test = get_oof(svm,x_train, y_train, x_test)

### Importancia de los features en base a los resultados obtenidos

In [10]:
rf_features = random_forest.feature_importances(x_train,y_train)

et_features = extra_trees.feature_importances(x_train, y_train)

ab_features = ada_boost.feature_importances(x_train, y_train)

gb_features = grad_boost.feature_importances(x_train,y_train)

# Creo que deberiamos agregar svm_features = ...

[0.07031283 0.18288966 0.38535799 0.36143952]
[0.24012803 0.08902858 0.50766786 0.16317553]
[0.03 0.33 0.53 0.11]
[0.09563239 0.30288287 0.28617408 0.31531066]


In [3]:
# ACA HAY QUE COPIAR Y PEGAR LA SALIDA DE ARRIBA PERO EN FORMATO ARRAY

### Creación de un dataframe con la importancia de los features para cada clasificador

In [11]:
columnas = np.array(['sexo', 'orden_estudio', 'tipo_de_trabajo', 'nivel_laboral'])

df_features = pd.DataFrame({'Features': columnas,
                            'Random Forest': rf_features,
                            'Extra Trees': et_features,
                            'Adaptive Boost': ab_features,
                            'Gradient Boost': gb_features
                            })

df_features = df_features[['Features', 'Random Forest', 'Extra Trees', 'Adaptive Boost', 'Gradient Boost']]

df_features.head()

,Features,Random Forest,Extra Trees,Adaptive Boost,Gradient Boost
0,sexo,None,None,None,None
1,orden_estudio,None,None,None,None
2,tipo_de_trabajo,None,None,None,None
3,nivel_laboral,None,None,None,None


Visualización de la importancia de los features para cada clasificador

TIRAR ACA UN PAR DE PLOTS

Agregamos una columna contenedora del promedio de la importancia de los features de cada fila

In [12]:
df_features['promedio'] = df_features.mean(axis=1)
df_features.head()

,Features,Random Forest,Extra Trees,Adaptive Boost,Gradient Boost,promedio
0,sexo,None,None,None,None,NaN
1,orden_estudio,None,None,None,None,NaN
2,tipo_de_trabajo,None,None,None,None,NaN
3,nivel_laboral,None,None,None,None,NaN


Visualización del promedio de la importancia los features

TIRAR ACA UN PAR DE PLOTS

# Modelos de Segundo Nivel

Utilizamos las predicciones obtenidas en el modelo de primer nivel como input del siguiente modelo.

Primero, analizamos las predicciones del primer nivel para cada clasificador.

In [13]:
predicciones_primer_nivel = pd.DataFrame({'Random Forest': rf_oof_train.ravel(),
                                          'Extra Trees': et_oof_train.ravel(),
                                          'Adaptive Boosting': ab_oof_train.ravel(),
                                          'Gradient Boosting': gb_oof_train.ravel()
                                          })
predicciones_primer_nivel.head()

,Adaptive Boosting,Extra Trees,Gradient Boosting,Random Forest
0,1.0,1.0,1.0,1.0
1,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,1.0,1.0,1.0


### Concatenamos las predicciones obtenidas en el primer nivel

In [14]:
x_train_2 = np.concatenate((et_oof_train, rf_oof_train, ab_oof_train, gb_oof_train, svm_oof_train), axis=1)

x_test_2 = np.concatenate((et_oof_test, rf_oof_test, ab_oof_test, gb_oof_test, svm_oof_test), axis=1)

### Aplicamos XGBoost para el modelo de segundo nivel

In [60]:
# Creamos el xgboost
'''
xgboost = xgb.XGBClassifier(
                            learning_rate = 0.02,
                            n_estimators= 1000,
                            max_depth= 5,
                            min_child_weight= 2,
                            gamma=0.9,                        
                            subsample=0.7,
                            colsample_bytree=0.7,
                            objective= 'binary:logistic',
                            nthread= -1,
                            scale_pos_weight=1)

# Lo entrenamos con el set de entrenamiento obtenido en el modelo de primer nivel
xgboost.fit(x_train_2, y_train)

# Realizamos las predicciones con el set definitivo
pred_final = gbm.predict(x_test_2)
'''

"\n# Creamos el xgboost\nxgboost = xgb.XGBClassifier(\n                            learning_rate = 0.02,\n                            n_estimators= 1000,\n                            max_depth= 5,\n                            min_child_weight= 2,\n                            gamma=0.9,                        \n                            subsample=0.7,\n                            colsample_bytree=0.7,\n                            objective= 'binary:logistic',\n                            nthread= -1,\n                            scale_pos_weight=1)\n\n# Lo entrenamos con el set de entrenamiento\nxgboost.fit(x_train_2, y_train)\n\n# Realizamos las predicciones con el set de pruebas\npredicciones_segundo_nivel = gbm.predict(x_test_2)\n"

In [16]:
from sklearn.linear_model import Perceptron

perceptron = Perceptron()
perceptron.fit(x_train_2, y_train)

pred_final = perceptron.predict(x_test_2)

# Submit File

Luego de haber entrenado y ajustado nuestros modelos de primer y segundo nivel, predecimos el test file definitivo correspondiente a la competencia de Kaggle.

In [17]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})

submit.to_csv('submit_ensamble.csv', index=False)